<a href="https://colab.research.google.com/github/Sudhir22/Spark-Notebooks/blob/main/MLlib_frequent_itemsets.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Now we authenticate a Google Drive client to download the file we will be processing in our Spark job.



In [2]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
id='1dhi1F78ssqR8gE6U-AgB80ZW7V_9snX4'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('products.csv')

id='1KZBNEaIyMTcsRV817us6uLZgm-Mii8oU'
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('order_products__train.csv')

If you executed the cells above, you should be able to see the dataset we will need for this Colab under the "Files" tab on the left panel.

In [28]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.ml.fpm import FPGrowth

Let's initialize the Spark context.

In [5]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

You can easily check the current version and get the link of the web interface. In the Spark UI, you can monitor the progress of your job and debug the performance bottlenecks (if your Colab is running with a **local runtime**).

In [6]:
spark

### Your task

If you run successfully the setup stage, you are ready to work with the **3 Million Instacart Orders** dataset. In case you want to read more about it, check the [official Instacart blog post](https://tech.instacart.com/3-million-instacart-orders-open-sourced-d40d29ead6f2) about it, a concise [schema description](https://gist.github.com/jeremystan/c3b39d947d9b88b3ccff3147dbcf6c6b) of the dataset, and the [download page](https://www.instacart.com/datasets/grocery-shopping-2017).

In this Colab, we will be working only with a small training dataset (~131K orders) to perform fast Frequent Pattern Mining with the FP-Growth algorithm.

In [7]:
products = spark.read.csv('products.csv', header=True, inferSchema=True)
orders = spark.read.csv('order_products__train.csv', header=True, inferSchema=True)

In [8]:
products.printSchema()

root
 |-- product_id: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- aisle_id: string (nullable = true)
 |-- department_id: string (nullable = true)



In [9]:
orders.printSchema()

root
 |-- order_id: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- add_to_cart_order: integer (nullable = true)
 |-- reordered: integer (nullable = true)



Using the Spark Dataframe API to join 'products' and 'orders', so that we will be able to see the product names in each transaction (and not only their ids).  Then, grouping by the orders by 'order_id' to obtain one row per basket (i.e., set of products purchased together by one customer). 

In [10]:
# YOUR CODE HERE
joined_data = orders.join(products,on=['product_id'],how='left')

In [33]:
grouped_data = joined_data.groupby(['order_id']).agg(collect_set('product_name').alias("product_set"))
grouped_data.show()

+--------+--------------------+
|order_id|         product_set|
+--------+--------------------+
|    1342|[Raw Shrimp, Seed...|
|    1591|[Cracked Wheat, S...|
|    4519|[Beet Apple Carro...|
|    4935|             [Vodka]|
|    6357|[Globe Eggplant, ...|
|   10362|[Organic Baby Spi...|
|   19204|[Reduced Fat Crac...|
|   29601|[Organic Red Onio...|
|   31035|[Organic Cripps P...|
|   40011|[Organic Baby Spi...|
|   46266|[Uncured Beef Hot...|
|   51607|[Donut House Choc...|
|   58797|[Concentrated But...|
|   61793|[Raspberries, Gre...|
|   67089|[Original Tofurky...|
|   70863|[Extra Hold Non-A...|
|   88674|[Organic Coconut ...|
|   91937|[No. 485 Gin, Mon...|
|   92317|[Red Vine Tomato,...|
|   99621|[Organic Baby Aru...|
+--------+--------------------+
only showing top 20 rows



In this Colab we will explore [MLlib](https://spark.apache.org/mllib/), Apache Spark's scalable machine learning library. Specifically, you can use its implementation of the [FP-Growth](https://spark.apache.org/docs/latest/ml-frequent-pattern-mining.html#fp-growth) algorithm to perform efficiently Frequent Pattern Mining in Spark.



In [34]:
# YOUR CODE HERE
fpGrowth = FPGrowth(itemsCol="product_set",minSupport=0.01,minConfidence=0.5)
model = fpGrowth.fit(grouped_data)


Compute how many frequent itemsets and association rules were generated by running FP-growth.


In [ ]:
model.freqItemsets.count()

In [ ]:
model.associationRules.count()